<div id='id0' > </div>

<body>
    <h1> <b> Análisis Exploratorio de Datos </b> </h1> 
    <h2> <b> Índice </b> </h2>
</body>

1.  [Generalidades](#id1)
2.  [Características Preliminares de las Fuentes de Datos](#id2)
3.  [Imoportación de la Librerías Requeridas](#id3)
4.  [Extración de los Datos](#id4)
5.  [Carga de los Datos en Dataframes](#id5)
6.  [Fuente de Datos: Bibliotecas Populares](#id6)
7.  [Fuente de Datos: Museos](#id7)
8.  [Fuente de Datos: Salas de Cines](#id8)
9.  [Unión de los Dataframes](#id9)
10. [Obtención de los Insights](#id10)

---


<div id='id1' > </div>

<body>
     <h1> <b> Generalidades </b> <br> </h1>
     <p> Los datos fueron provistos en archivos de diferentes extensiones, como csv o json. La información ha de ser procesada y normalizada para cada dataset, entre otras tareas, para obtener información relevante, tal como: <p>
     <ul>
          <li> <p> 
          Máxima duración según tipo de film (película/serie), por plataforma y por año: El request debe ser: get_max_duration(año, plataforma, [min o season])</b>. </li>
          <li> <p> 
          Cantidad de películas y series (separado) por plataforma El request debe ser: get_count_plataform(plataforma)</b>. </li>
          <li> <p> 
          Cantidad de veces que se repite un género y plataforma con mayor frecuencia del mismo. El request debe ser: get_listedin('genero')</b>. </li>
          <li> <p> 
          Actor que más se repite según plataforma y año. El request debe ser: get_actor(plataforma, año)</b>. </li>
     </ul> 
</body>

---


<div id='id2' > </div>

<head>
     <meta http-equiv="Content-Type"content="text/html"charset="utf-8">
     <link href="./css_estilos/estilo_01.css"rel="stylesheet"type="text/css"> 
</head>
<body>
     <h1> <b> Características Preliminares de las Fuentes de Datos </b> <br> </h1>
     <p>
     Los datasets disponibles corresponden ha los titulos disponibles en cuatro proveedores de streaming. <br>
     <br>
     Los recursos recuperados de las fuentes fueron:
          <ul>
               <li> <p> 
               Títulos de amazon prime: amazon_prime_titles.csv</b>. </li>
               <li> <p> 
               Títulos de Disney Plus: disney_plus_titles.csv</b>. </li>
               <li> <p> 
               Títulos de Hulu: hulu_titles.csv</b>. </li>
               <li> <p> 
               Títulos de Netflix: netflix_titles.json</b>. </li>
               </ul> 
     </p>
</body>

[ Índice </p> ](#id0)

---

<div id='id3' > </div>

<head>
     <meta http-equiv="Content-Type"content="text/html"charset="utf-8">
     <link href="./css_estilos/estilo_01.css"rel="stylesheet"type="text/css"> 
</head>
<body>
     <h1> <b> Imoportación de la Librerías Requeridas </b> <br> </h1>
</body>

In [2]:
#   Importamos la librerías
#Python 
import os
from pathlib import Path
from datetime import datetime

#Request
import requests

#Pandas
import pandas as pd

[ Índice </p> ](#id0)

---

<div id='id4' > </div>

<head>
     <meta http-equiv="Content-Type"content="text/html"charset="utf-8">
     <link href="./css_estilos/estilo_01.css"rel="stylesheet"type="text/css"> 
</head>
<body>
     <h1> <b> Extración de los Datos </b> <br> </h1>
</body>

In [3]:
#   Definimos los directorios y las rutas
MAIN_DIR = (Path(os.path.abspath(''))).parent                                    #out: "$PWD/streaming_catalog_api"
DATA_DIR = Path(f"{MAIN_DIR}/data")                                              #out: "$PWD/streaming_catalog_api/data"
EXT_DIR =  Path(f"{DATA_DIR}/external")                                          #out: "$PWD/streaming_catalog_api/data/external"
INT_DIR = Path(f"{DATA_DIR}/interim")                                            #out: "$PWD/streaming_catalog_api/data/interim"
PRO_DIR = Path(f"{MAIN_DIR}/processed")                                          #out: "$PWD/streaming_catalog_api/data/raw"

In [5]:
print(
    "MAIN_DIR:" f"{MAIN_DIR}\n",
    "DATA_DIR:" f"{DATA_DIR}\n",
    "EXT_DIR:" f"{EXT_DIR}\n",
    "INT_DIR:" f"{INT_DIR}\n",
    "PRO_DIR:" f"{PRO_DIR}\n"
)

MAIN_DIR:c:\Users\Jose Salas\OneDrive\Escritorio\streaming_catalog_api
 DATA_DIR:c:\Users\Jose Salas\OneDrive\Escritorio\streaming_catalog_api\data
 EXT_DIR:c:\Users\Jose Salas\OneDrive\Escritorio\streaming_catalog_api\data\external
 INT_DIR:c:\Users\Jose Salas\OneDrive\Escritorio\streaming_catalog_api\data\interim
 PRO_DIR:c:\Users\Jose Salas\OneDrive\Escritorio\streaming_catalog_api\processed



In [4]:
#   Creamos una lista de diccionarios para almacenar la categoría y la url
category_list = [
    {
        "name": "amazon_prime_titles",
        "url": "https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/amazon_prime_titles.csv"
    },
    {
        "name": "disney_plus_titles",
        "url": "https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/disney_plus_titles.csv"
    },
    {
        "name": "hulu_titles",
        "url": "https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/hulu_titles.csv"
    },
    {
        "name": "netflix_titles",
        "url": "https://raw.githubusercontent.com/HX-FAshur/PI01_DATA05/main/Datasets/netflix_titles.json"
    }
    ]

In [9]:
category_list[3]["name"]

'netflix_titles'

In [15]:
def get_data(category_list):
    for i in range(len(category_list)):
        streaming_plataform = category_list[i]["name"]
        if streaming_plataform != "netflix_titles":
            file_name = "{category}/{year}-{month:02d}/{category}-{day:02d}-{month:02d}-{year}.csv"
        else:
            file_name = "{category}/{year}-{month:02d}/{category}-{day:02d}-{month:02d}-{year}.json"
        
        now = datetime.now()
        category = category_list[i]["name"]
        year = now.year
        month = now.month
        day = now.day
        
        r = requests.get(category_list[i]["url"])
        r.encoding = 'utf-8'
        FILE_NAME = file_name.format(
            category = category, 
            year = year, 
            month = month, 
            day = day
            )

        FILE_PATH = EXT_DIR/FILE_NAME
        FILE_PATH.parent.mkdir(
            parents=True,
            exist_ok=True
            )
            
        with open(FILE_PATH,'w') as file_outp:
            file_outp.write(r.text)
            print(f"Se ha guardado el archivo en el directorio: {FILE_PATH} \n")

In [16]:
get_data(category_list = category_list)

Se ha guardado el archivo en el directorio: c:\Users\Jose Salas\OneDrive\Escritorio\streaming_catalog_api\data\external\amazon_prime_titles\2022-12\amazon_prime_titles-08-12-2022.csv 

Se ha guardado el archivo en el directorio: c:\Users\Jose Salas\OneDrive\Escritorio\streaming_catalog_api\data\external\disney_plus_titles\2022-12\disney_plus_titles-08-12-2022.csv 

Se ha guardado el archivo en el directorio: c:\Users\Jose Salas\OneDrive\Escritorio\streaming_catalog_api\data\external\hulu_titles\2022-12\hulu_titles-08-12-2022.csv 

Se ha guardado el archivo en el directorio: c:\Users\Jose Salas\OneDrive\Escritorio\streaming_catalog_api\data\external\netflix_titles\2022-12\netflix_titles-08-12-2022.json 

